In [30]:
from spektral.datasets import citation
from spektral.layers import GraphConv # for Graph Convolutional Neural Network
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout
import pandas as pd

In [2]:
A, X, y, train_mask, val_mask, test_mask = citation.load_data('cora')
# A: adjacency matrix == shape(N,N)
# X: node features == shape(N,F)
# y: labels == shape(N,n_classes)
# train_mask, val_mask, test_mask: boolean, shows which nodes belong to training, validation, testing sets

Loading cora dataset
Pre-processing node features


In [3]:
N = A.shape[0]
F = X.shape[-1]
n_classes = y.shape[-1]

In [4]:
# Create GNN

X_in = Input(shape=(F,))
A_in = Input(shape=(N,), sparse=True)

X_1 = GraphConv(16, 'relu')([X_in, A_in])
X_1 = Dropout(.5)(X_1)
X_2 = GraphConv(n_classes, 'softmax')([X_1, A_in])

model = Model(inputs=[X_in, A_in], outputs=X_2)

An important thing to notice at this point is how we defined the Input layers of our model. Because the "elements" of our dataset are the nodes themselves, we are telling Keras to consider each node as a separate sample so that the batch axis is implicitly defined as None.
In other words, a sample of the node attributes will be a vector of shape (F, ) and a sample of the adjacency matrix will be one row of shape (N, ).

# Training GCN

When training GCN, we have to pre-process the adjacency matrix to 1) add self-loops and 2) scale the weights of a node's connections according to its degree.

Some layers in Spektral require a different type of pre-processing in order to work correctly, and some work out-of-the-box on the binary A. The pre-processing required by each layer is available as a static class method preprocess().

In [5]:
A = GraphConv.preprocess(A).astype('f4')

In [6]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             weighted_metrics=['acc']) # weighted metrics instead of metrics due to particular semi-supervised problem
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1433)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2708)]       0                                            
__________________________________________________________________________________________________
graph_conv (GraphConv)          (None, 16)           22944       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 16)           0           graph_conv[0][0]             

In [7]:
# prep data
X = X.toarray()
A = A.astype('f4')
validation_data = ([X, A], y, val_mask)

X, A

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 <2708x2708 sparse matrix of type '<class 'numpy.float32'>'
 	with 13264 stored elements in Compressed Sparse Row format>)

In [8]:
X.shape, A.shape, validation_data

((2708, 1433),
 (2708, 2708),
 ([array([[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
   <2708x2708 sparse matrix of type '<class 'numpy.float32'>'
   	with 13264 stored elements in Compressed Sparse Row format>],
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 1, 0, 0],
         [0, 0, 0, ..., 1, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=int32),
  array([False, False, False, ..., False, False, False])))

We have set batch_size=N and shuffle=False. This is because the default behaviour of Keras is to split the data into batches of 32 and shuffle the samples at each epoch. However, shuffling the adjacency matrix along one axis and not the other means that row i will represent a different node than column i.
At the same time, if we split the graph into batches we may end up in a situation where we need to use a node attribute that is not part of the batch. The only solution is to take all the node features at the same time, hence batch_size=N.

In [22]:
# train model
model.fit([X, A], y,
         sample_weight=train_mask,
         validation_data=validation_data,
         batch_size=N,
         shuffle=False,
         epochs=50)

Epoch 1/50
1/1 [==============================] - 0s 168ms/step - loss: 0.1006 - acc: 0.2143 - val_loss: 0.3592 - val_acc: 0.2000
Epoch 2/50
1/1 [==============================] - 0s 169ms/step - loss: 0.1005 - acc: 0.2214 - val_loss: 0.3591 - val_acc: 0.2080
Epoch 3/50
1/1 [==============================] - 0s 175ms/step - loss: 0.1005 - acc: 0.2929 - val_loss: 0.3591 - val_acc: 0.2200
Epoch 4/50
1/1 [==============================] - 0s 167ms/step - loss: 0.1005 - acc: 0.3143 - val_loss: 0.3590 - val_acc: 0.2420
Epoch 5/50
1/1 [==============================] - 0s 176ms/step - loss: 0.1005 - acc: 0.2643 - val_loss: 0.3589 - val_acc: 0.2600
Epoch 6/50
1/1 [==============================] - 0s 197ms/step - loss: 0.1004 - acc: 0.3357 - val_loss: 0.3588 - val_acc: 0.3000
Epoch 7/50
1/1 [==============================] - 0s 173ms/step - loss: 0.1004 - acc: 0.3714 - val_loss: 0.3587 - val_acc: 0.3420
Epoch 8/50
1/1 [==============================] - 0s 170ms/step - loss: 0.1003 - acc: 0.34

In [21]:
pd.DataFrame([train_mask, val_mask, test_mask]).T.head(20)

,0,1,2
0,True,False,False
1,True,False,False
2,True,False,False
3,True,False,False
4,True,False,False
5,True,False,False
6,True,False,False
7,True,False,False
8,True,False,False
9,True,False,False


In [25]:
# Evaluate the model
eval_results = model.evaluate([X, A],
                              y,
                              sample_weight=test_mask,
                              batch_size=N)

1/1 [==============================] - 0s 733us/step - loss: 0.7066 - acc: 0.6810


In [26]:
print('Done.\n'
      'Test loss: {}\n'
      'Test accuracy: {}'.format(*eval_results))

Done.
Test loss: 0.7065826654434204
Test accuracy: 0.6809999942779541
